# Bronze Layer Tables

## Orders_raw

In [0]:
CREATE OR REFRESH STREAMING LIVE TABLE orders_raw -- Auto Loader 
COMMENT "The raw books orders, ingested from orders-raw"
AS SELECT * FROM cloud_files("${dataset.path}/orders-raw", "parquet", 
                            map("schema", "order_id STRING, order_timestamp LONG, customer_id STRING, quantity LONG"))

## Customers

In [0]:
CREATE OR REFRESH STREAMING LIVE TABLE customers
COMMENT "The customers lookup table, ingested from customers-json"
AS SELECT * FROM json.`${dataset.path}/customers-json`

# Silver Layer Tables

## Orders_cleaned

In [0]:
CREATE OR REFRESH STREAMING LIVE TABLE orders_cleaned (
  CONSTRAINT valid_order_number EXPECT (order_id IS NOT NULL) ON VIOLATION DROP ROW
)
COMMENT "The cleaned books orders with valid order_id"
AS 
  SELECT order_id, quatity, o.customer_id, c.profile:first_name as f_name, c.profile:last_name as l_name,
         cast(from_unixtime(order_timestamp, 'yyyy-MM-dd HH:mm:ss') as timestamp) as order_timestamp,
         c.profile:address:country as country
  FROM STREAM(LIVE.orders_raw) o
  LEFT JOIN LIVE.customers c
  ON o.customer_id = c.customer_id

# Gold tables

In [0]:
CREATE OR REFRESH STREAMING LIVE TABLE cn_daily_customer_books
COMMENT "Daily number of books per customer in China"
AS
  SELECT customer_id, f_name, l_name, date_trunc("DD", order_timestamp) order_date, sum(quantity) as book_counts
FROM STREAM(LIVE.orders_cleaned)
WHERE country = 'China'
GROUP BY customer_id, f_name, l_name, date_trunc("DD", order_timestamp)